In [12]:
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.cross_validation import train_test_split

import xgboost as xgb

dtr = xgb.XGBRegressor(max_depth=10, learning_rate=0.1)

pd.options.mode.chained_assignment = None  # default='warn'

train_list = ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
        '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
        '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
        '180701', '180702', '180703', '180704', '180705', '180706']

valid_list = ['180707', '180708']

test_list = ['180709', '180710', '180711', '180712', '180713', '180714', '180715']

day_all = ['171211', '171212', '171213', '171214', '171215', '171216', '171217', '171218', '171219', '171220', \
           '171221', '171222', '171223', '171224', '171225', '171226', '171227', '171228', '171229', '171230', \
           '171231', '180101', '180102', '180104', '180105', '180106', '180107', '180108', '180109', '180110', \
           '180111', '180112', '180113', '180114', '180115', '180116', '180117', '180118', '180119', '180120', \
           '180121', '180122', '180123', '180124', '180125', '180126', '180127', '180128', '180129', '180130', '180131',\
           '180201', '180202', '180203', '180204', '180205', '180206', '180207', '180208', '180209', '180210',\
           '180211', '180212', '180213', '180214', '180215', '180216', '180217', '180218', '180219', '180220', \
           '180221', '180222', '180223', '180224', '180225', '180226', '180227', '180228', \
           '180301', '180302', '180303', '180304', '180305', '180306', '180307', '180308', '180309', '180310', \
           '180311', '180312', '180313', '180314', '180315', '180316', '180317', '180318', '180319', '180320',\
           '180321', '180322', '180323', '180324', '180325', '180326', '180327', '180328', '180329', '180330', '180331', \
           '180401', '180402', '180403', '180404', '180405', '180406', '180407', '180408', '180409', '180410', \
           '180411', '180412', '180413', '180414', '180415', '180416', '180417','180418', '180419', '180420',\
           '180421', '180422', '180423', '180424', '180425', '180426', '180427', '180428', '180429', '180430',\
           '180501', '180502', '180503', '180504', '180505', '180506', '180507', '180508', '180509', '180510',\
           '180511', '180512', '180513', '180514', '180515', '180516', '180517', '180518', '180519', '180520',\
           '180521', '180522', '180523', '180524', '180525', '180526', '180527', '180528', '180529', '180530',\
           '180531','180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
           '180711', '180712', '180713', '180714', '180715','180716','180717']


In [2]:
#读取数据 按天

# 划分训练集， 验证集 测试集

train_df = []
for day in train_list:
    print(day,end=' ')   
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train/train_%s_edt.csv'%day)
    
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    
    #过滤结束日期超过2天的
    day_after_2 = day_all[day_all.index(day)+2]
    day_after_2_int = int('20'+day_after_2+'000000')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    
    train_df.append(day_all_data.sample(frac=0.1))
    
train = pd.concat(train_df, ignore_index=True)

valid_df = []
for day in valid_list:
    print(day,end=' ')   
    day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train/train_%s_edt.csv'%day)
    #day_all_data = day_all_data[day_all_data.end_time.notnull()]
    day_after_2 = day_all[day_all.index(day)+2]
    day_after_2_int = int('20'+day_after_2+'000000')
    day_all_data = day_all_data[day_all_data.end_time<day_after_2_int]
    
    valid_df.append(day_all_data)
    
valid = pd.concat(valid_df, ignore_index=True)

# test_df = []

# for day in test_list:
#     print(day,end=' ')   
#     day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train/train_%s_edt.csv'%day)
#     day_all_data = day_all_data[day_all_data.end_time.notnull()]
#     test_df.append(day_all_data)
    
# test = pd.concat(test_df, ignore_index=True)


180602 180603 180604 180605 180606 180607 180608 180609 180610 180611 180612 180613 180614 180615 180616 180617 180618 180619 180620 180621 180622 180623 180624 180625 180626 180627 180628 180629 180630 180701 180702 180703 180704 180705 180706 180707 180708 

In [4]:


train = train.round(3)
valid = valid.round(3)


#过滤训练不需要的数据字段
cols_train =[clm for clm in valid.columns if clm not in ['order_id','job_name','v_date','order_d','from_time_1000_start',\
                                                            'start_time','end_time','order_id','job_name','day_0_bef_start_time',\
                                                            'day_0_bef_end_time']]


valid_x = valid[cols_train]
valid_y = valid['day_0_bef_end_time']

train_x,test_x,train_y,test_y = train_test_split(train[cols_train],train['day_0_bef_end_time'],random_state=33,test_size=0.2)

print(train_x.shape)
print(test_x.shape)
print(valid_x.shape)

(193399, 205)
(48350, 205)
(282678, 205)


In [ ]:
# 训练时不需要的特征
v_date	order_d	from_time_1000_start
start_time	end_time
order_id	job_name
day_0_bef_start_time	day_0_bef_end_time


In [14]:



dtr.fit(train_x,train_y)

valid_predict = dtr.predict(valid_x)

test_predict = dtr.predict(test_x)

print('valid:')
print(mean_absolute_error(valid_y,valid_predict))
print(mean_squared_error(valid_y,valid_predict))

print()
print('train:')
print(mean_absolute_error(test_y,test_predict))
print(mean_squared_error(test_y,test_predict))

valid:
87.8621624625
20347.1576495

train:
67.5874337466
17802.2401908


In [10]:
day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train\train_180707_edt.csv')
#day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train/train_%s_edt.csv')
day_all_data = day_all_data[day_all_data.end_time.notnull()]
test_predict = dtr.predict(day_all_data[cols_train])

print(mean_absolute_error(test_predict,day_all_data['day_0_bef_end_time']))
print(mean_squared_error(test_predict,day_all_data['day_0_bef_end_time']))

91.9653195614
24218.1828896


In [15]:
day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train\train_180708_edt.csv')
#day_all_data = pd.read_csv(r'E:\jupyter\CTM\valid_data\train/train_%s_edt.csv')
day_all_data = day_all_data[day_all_data.end_time.notnull()]

test_predict = dtr.predict(day_all_data[cols_train])

print(mean_absolute_error(test_predict,day_all_data['day_0_bef_end_time']))
print(mean_squared_error(test_predict,day_all_data['day_0_bef_end_time']))

day_all_data['predict_endt'] = test_predict

day_all_data.to_csv('error_anlaysis.csv')

84.2841424267
16865.4312558
